# Hidden markov model


In [ ]:
import datetime as dt
from typing import List, Dict, Optional

import numpy as np
import plotly.graph_objects as go
import polars as pl
from hmmlearn.hmm import GaussianHMM
import pandas as pd
import xgboost as xgb

from homelab_pipelines.utils.paths import Paths

In [ ]:
class Config:
    rv_rolling_rows: int = 8
    """Number of rows to consider for $RV_t$."""
    volsurp_window_days: int = 30
    """Number of days to consider for time-of-day mean of $VolSurp_t$."""


class HMMConfig:
    n_days_train: int = 26 * 7  # 6 months


class XGBoostConfig:
    n_days_train: int = 2 * 52 * 7  # Two years


## Loading data


In [ ]:
prices = pl.read_parquet(Paths.repo_root / "data" / "BTCUSDT.parquet")
prices

In [ ]:
prices.describe()

## Preprocessing data

For the HMM, we generate certain features:

- $RV_t$: realized volatility of the last $m$ bars
- $|r_t|$ where $r_t$ is the Intraday log-return. This means assets that are not open 24 hours a day should omit the first observation of each day. Crypto assets are open the entire day, making this part slightly easier.
- $\text{VolSurp}_t$: volume surprise relative to the time-of-day mean
- $\text{Range}_t$: intraday high-low range


In [ ]:
trading_day_start_time_seconds = 0
trading_day_duration_seconds = 24 * 60 * 60

X = (
    prices.sort("start_time_utc")
    .with_columns(
        log_range=pl.col("high").log() - pl.col("low").log(),
        sfm=(
            pl.col("start_time_utc") - pl.col("start_time_utc").dt.truncate("1d")
        ).dt.total_seconds(),
        # Log returns log(close_{t}) - log(close_{t-k})
        log_returns=pl.col("close").log().diff(1),
        log_returns_1=pl.col("close").log().diff(1),
        log_returns_4=pl.col("close").log().diff(4),
        # Log returns ahead
        # NOTE: we need to multiply with -1 such that we get log(close_{t+k}) - log(close_{t})
        log_returns_ahead_4=-pl.col("close").log().diff(-4),
    )
    .with_columns(
        realized_volatility=pl.col("log_returns")
        .pow(2)
        .rolling_sum(Config.rv_rolling_rows)
        .sqrt(),
        abs_log_returns=pl.col("log_returns").abs(),
        # NOTE: crypto markets are open 24/7. When using this for other assets, this may not be the case.
        trading_day_frac=(
            (pl.col("sfm") - trading_day_start_time_seconds).clip(
                0, trading_day_duration_seconds
            )
            / trading_day_duration_seconds
        ),
    )
    .with_columns(
        trading_day_sin=pl.col("trading_day_frac").sin(),
        trading_day_cos=pl.col("trading_day_frac").cos(),
    )
)

# Calculate rolling volatility at each time of day
X = X.join(
    X.with_columns(
        time_of_day=pl.col("start_time_utc").dt.time(),
    )
    .rolling(
        index_column="start_time_utc",
        period=f"{Config.volsurp_window_days}d",
        group_by="time_of_day",
    )
    .agg(
        volume_mean_time_of_day_rolling=pl.col("volume")
        .clip(
            lower_bound=pl.col("volume").quantile(0.05),
            upper_bound=pl.col("volume").quantile(0.95),
        )
        .mean()
    )
    .drop("time_of_day"),
    on="start_time_utc",
    how="inner",
)

# Since we based our rolling mean volatility based on the first k days, we omit the first k days
X = X.filter(
    pl.col("start_time_utc")
    >= pl.col("start_time_utc").min() + dt.timedelta(days=Config.volsurp_window_days)
)

# Calculate VolSurp
X = X.with_columns(
    volsurp=pl.col("volume").log() - pl.col("volume_mean_time_of_day_rolling").log()
)
X = X.sort("start_time_utc")
X

## Defining the model

The model consists of two stages. First, we use a hidden markov model to estimate regimes. For each row, this gives us state probabilities and the expected duration within the current state. This is used as input for the ML model.


In [ ]:
class RegimeSwitchingModel:
    def __init__(self, **kwargs) -> None:
        self.hmm = GaussianHMM(**kwargs)
        self._fitted_columns: Optional[List[str]] = None

    def fit(self, X: pd.DataFrame) -> None:
        self._fitted_columns = X.columns.to_list()
        self.hmm.fit(X.to_numpy())

    def predict(self, X: pd.DataFrame) -> pd.DataFrame:
        X_columns_list = X.columns.to_list()

        if self._fitted_columns is None:
            raise ValueError("This instance should be fitted before calling predict")
        elif X_columns_list != self._fitted_columns:
            raise ValueError(
                f"This model has been fitted on columns {self._fitted_columns}, but received {X_columns_list}"
            )

        gamma = self.hmm.predict_proba(X.to_numpy())
        persistence = 1 / (1 - np.diag(self.hmm.transmat_))

        result = pd.DataFrame(
            gamma,
            columns=[f"regime_gamma_{i}" for i in range(gamma.shape[-1])],
            index=X.index,
        )
        result["regime_persistence"] = gamma @ persistence
        return result


class XGBoostQuantileRegressor:
    def __init__(self, quantiles: List[float], **kwargs) -> None:
        if "objective" in kwargs or "quantile_alpha" in kwargs:
            raise ValueError(
                "Model kwargs may not contain the objective or quantile_alpha"
            )

        self.quantile_models: Dict[str, xgb.XGBRegressor] = {
            f"p{q * 100:.0f}": xgb.XGBRegressor(
                objective="reg:quantileerror", quantile_alpha=q, **kwargs
            )
            for q in quantiles
        }
        self._fitted_columns: Optional[List[str]] = None

    def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        self._fitted_columns = X.columns.to_list()

        X_np = X.to_numpy()
        y_np = y.to_numpy()

        for _, model in self.quantile_models.items():
            model.fit(X_np, y_np)

    def predict(self, X: pd.DataFrame) -> pd.DataFrame:
        X_columns_list = X.columns.to_list()

        if self._fitted_columns is None:
            raise ValueError("This instance should be fitted before calling predict")
        elif X_columns_list != self._fitted_columns:
            raise ValueError(
                f"This model has been fitted on columns {self._fitted_columns}, but received {X_columns_list}"
            )

        X_np = X.to_numpy()

        result = pd.DataFrame(
            {
                quantile_str: model.predict(X_np)
                for quantile_str, model in self.quantile_models.items()
            },
            index=X.index,
        )
        return result


In [ ]:
X_train = X.filter(
    pl.col("start_time_utc") < pl.col("start_time_utc").max() - dt.timedelta(weeks=1)
).sort("start_time_utc")
X_test = X.filter(
    pl.col("start_time_utc") >= pl.col("start_time_utc").max() - dt.timedelta(weeks=1)
).sort("start_time_utc")

print(
    f"X_train range: [{X_train['start_time_utc'].min()}, {X_train['start_time_utc'].max()}]"
)
print(
    f"X_test range: [{X_test['start_time_utc'].min()}, {X_test['start_time_utc'].max()}]"
)

In [ ]:
X_train_hmm = (
    X_train.filter(
        pl.col("start_time_utc")
        >= pl.col("start_time_utc").max() - dt.timedelta(weeks=26)  # last 6 months
    )
    .select(
        "start_time_utc",
        "abs_log_returns",
        "log_range",
        "realized_volatility",
        "volsurp",
    )
    .to_pandas()
    .set_index("start_time_utc")
)
X_train_hmm

In [ ]:
hmm_model = RegimeSwitchingModel(
    n_components=3,  # 3–4 regimes
    covariance_type="full",
    n_iter=500,
    tol=1e-4,
    init_params="stmc",  # startprob, transmat, means, covars
    random_state=42,
)
hmm_model.fit(X_train_hmm)

hmm_output_train = hmm_model.predict(
    X_train.select(
        "start_time_utc",
        "abs_log_returns",
        "log_range",
        "realized_volatility",
        "volsurp",
    )
    .to_pandas()
    .set_index("start_time_utc")
)
hmm_output_train

In [ ]:
X_train_xgb_polars = pl.concat(
    [X_train, pl.from_pandas(hmm_output_train, include_index=True)], how="align_left"
).filter(
    pl.col("start_time_utc") >= pl.col("start_time_utc").max() - dt.timedelta(days=365)
)

response_variable_name = "log_returns_ahead_4"
features: List[str] = [
    "log_returns_4",
    "volume",
    "trading_day_sin",
    "trading_day_cos",
    "log_range",
    "realized_volatility",
    "volsurp",
    *[col for col in X_train_xgb_polars.columns if col.startswith("regime_")],
]

X_train_xgb = (
    X_train_xgb_polars.select(
        "start_time_utc",
        response_variable_name,
        *features,
    )
    .to_pandas()
    .set_index("start_time_utc")
)


y_train_xgb = X_train_xgb[response_variable_name]
X_train_xgb = X_train_xgb.drop(response_variable_name, axis=1)

X_train_xgb

In [ ]:
model = XGBoostQuantileRegressor(quantiles=[0.15, 0.5, 0.85], random_state=42)
model.fit(X_train_xgb, y_train_xgb)

y_train_predictions = model.predict(X_train_xgb)
y_train_predictions

### Forecasting


In [ ]:
# Generate HMM output
X_test_hmm = (
    X_test.select(
        "start_time_utc",
        "abs_log_returns",
        "log_range",
        "realized_volatility",
        "volsurp",
    )
    .to_pandas()
    .set_index("start_time_utc")
)
hmm_output_test = hmm_model.predict(X_test_hmm)

# Create test dataset for ML model
X_test_xgb_polars = pl.concat(
    [X_test, pl.from_pandas(hmm_output_test, include_index=True)],
    how="align_left",
)

X_test_xgb = (
    X_test_xgb_polars.select(
        "start_time_utc",
        *features,
    )
    .to_pandas()
    .set_index("start_time_utc")
)


# Create forecasts
predictions = model.predict(X_test_xgb)
predictions

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=X_test["start_time_utc"],
        y=X_test[response_variable_name],
        name=response_variable_name,
    ),
)

for col in predictions.columns:
    fig.add_trace(
        go.Scatter(
            x=predictions.index,
            y=predictions[col],
            name=col,
            line_color=f"rgba(128, 0, 128, {'0.2' if col != 'p50' else '1'})",
        )
    )

lower_column = min(predictions.columns)
upper_column = max(predictions.columns)

# Upper bound
fig.add_trace(
    go.Scatter(
        x=predictions.index,
        y=predictions[upper_column],
        mode="lines",
        line=dict(width=0),
        name="Upper bound",
        showlegend=False,
        hoverinfo="skip",
    )
)

# Lower bound (fills to previous trace)
fig.add_trace(
    go.Scatter(
        x=predictions.index,
        y=predictions[lower_column],
        mode="lines",
        line=dict(width=0),
        fill="tonexty",
        fillcolor="rgba(128, 0, 128, 0.2)",
        name="Prediction interval",
        showlegend=False,
        hoverinfo="skip",
    )
)


fig.update_layout(
    dict(
        title="Log returns (t+k)",
        hovermode="x",
        xaxis=dict(
            showline=True,
            showspikes=True,
            spikemode="across",
            spikesnap="cursor",
            spikedash="solid",
            spikecolor="black",
            spikethickness=1,
        ),
    )
)

fig.show()

In [ ]:
predicted_close_prices = pl.concat(
    [
        X_test,
        pl.from_pandas(predictions, include_index=True),
    ],
    how="align_full",
).with_columns(
    *[
        (pl.col("close") * pl.col(col).exp()).shift(4).alias(f"close_{col}")
        for col in predictions.columns
    ]
)
predicted_close_prices = predicted_close_prices.select(
    "start_time_utc",
    *[col for col in predicted_close_prices.columns if col.startswith("close")],
)
predicted_close_prices

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=predicted_close_prices["start_time_utc"],
        y=predicted_close_prices["close"],
        name="close",
    ),
)

columns_without_actuals = [
    col
    for col in predicted_close_prices.columns
    if col not in ("start_time_utc", "close")
]

for col in columns_without_actuals:
    fig.add_trace(
        go.Scatter(
            x=predicted_close_prices["start_time_utc"],
            y=predicted_close_prices[col],
            name=col,
            line_color=f"rgba(128, 0, 128, {'1' if col.endswith('p50') else '0.2'})",
        )
    )

lower_column = min(columns_without_actuals)
upper_column = max(columns_without_actuals)

# Upper bound
fig.add_trace(
    go.Scatter(
        x=predicted_close_prices["start_time_utc"],
        y=predicted_close_prices[upper_column],
        mode="lines",
        line=dict(width=0),
        name="Upper bound",
        showlegend=False,
        hoverinfo="skip",
    )
)

# Lower bound (fills to previous trace)
fig.add_trace(
    go.Scatter(
        x=predicted_close_prices["start_time_utc"],
        y=predicted_close_prices[lower_column],
        mode="lines",
        line=dict(width=0),
        fill="tonexty",
        fillcolor="rgba(128, 0, 128, 0.2)",
        name="Prediction interval",
        showlegend=False,
        hoverinfo="skip",
    )
)


fig.update_layout(
    dict(
        title="Close price vs predicted",
        hovermode="x",
        xaxis=dict(
            showline=True,
            showspikes=True,
            spikemode="across",
            spikesnap="cursor",
            spikedash="solid",
            spikecolor="black",
            spikethickness=1,
        ),
    )
)

fig.show()
